In [1]:
from xml.dom.minidom import parse, parseString
import pandas as pd
import geopy.distance
import numpy as np

In [2]:
FORMAT = "%Y-%m-%dT%H:%M:%SZ"   # the format for the date
FILE_NAME = "Coronatour Mortirolo"

# the time to start the ride
begin_date = pd.to_datetime("2020-03-24 15:00:00")
# one second
delta = pd.Timedelta(10**9)
# the speed at which the ride is done
v_norm = 40/3.6

# functions to calculate the (estimate of) speed as a function of slope s
def v_pos(s, k=0.2, v_min=v_norm):
    return v_min/(1+k*s)
def v_neg(s, l=15, v_min=v_norm):
    return v_min + l*np.sqrt(-s)

In [3]:
# extract the xml data from the gpx and all trackpoints
xml = parse(FILE_NAME+".gpx")
trackpoints = xml.getElementsByTagName("trkpt")

# add the start time to the first point
node = trackpoints[0]              # get the first trackpoint
newEle = xml.createElement("time") # create a new element to hold the time
newText= xml.createTextNode(begin_date.strftime(FORMAT))      # create node with the date
newEle.appendChild(newText);       # add this node to the new element
node.appendChild(newEle)           # add the time element to the trackpoint

time = begin_date
height = float(node.childNodes[1].childNodes[0].nodeValue)

for i in range(1, len(trackpoints)):
    node = trackpoints[i]              # get the first trackpoint
    newEle = xml.createElement("time") # create a new element to hold the time
    
    # calculate the distance between this point and the previous
    point0 = trackpoints[i-1]
    point1 = trackpoints[i]
    # extract coordinates
    coord0 = (float(point0.getAttributeNode("lat").nodeValue), float(point0.getAttributeNode("lon").nodeValue))
    coord1 = (float(point1.getAttributeNode("lat").nodeValue), float(point1.getAttributeNode("lon").nodeValue))
    height0 = float(point0.childNodes[1].childNodes[0].nodeValue)
    height1 = float(point1.childNodes[1].childNodes[0].nodeValue)
    # get the distance in meter
    dist = geopy.distance.geodesic(coord0, coord1).m
    delta_h = height1-height0
    slope = 100*delta_h/dist    # calculate slope in % (multiply by 100)
    
    # calculate the time between this point and the previous and the time at this point
    speed = v_norm
    if delta_h <= 0:
        speed = v_neg(slope)
    else:
        speed = v_pos(slope)
    
    seconds = dist/speed
    time += seconds*delta
    
    newText= xml.createTextNode(time.strftime(FORMAT))      # create node with the date
    newEle.appendChild(newText);       # add this node to the new element
    node.appendChild(newEle)           # add the time element to the trackpoint
    
    
with open(FILE_NAME+"-with-time.gpx", "w") as file:
    print(xml.toprettyxml(), file=file)

In [4]:
node = trackpoints[0]

In [5]:
ele = node.childNodes[1]

In [6]:
ele.childNodes[0].nodeValue

'904.4300000000001'